# Анализ тональности отзывов
# Неделя 3 - соревнование на kaggle 


В этом проекте вам предстоит решать задачи анализа тональности текстов в нескольких разных постановках. Начнется все с несложных экспериментов на отзывах на фильмы, в ходе которых вы построите простую модель и немного доработаете ее. Затем вам предстоит поучаствовать в соревновании по сентимент-анализу отзывов на товары и сделать интерактивную демонстрацию для своего алгоритма, которую можно будет показать даже тем, кто никогда не видел Python и машинное обучение. После этого вы поупражняетесь в парсинге веб-страниц и столкнетесь с жестокой реальностью - к вам придет заказчик, который захочет от вас сентимент-анализ отзывов на определенную категорию товаров, но вот выборку для обучения придется собирать самим. Как и угадывать, что же заказчик называет негативными отзывами, а что позитивными. В конце проекта вам потребуется сделать демонстрацию и для этого алгоритма, чтобы заказчик мог с ним поиграться, а не только лишь ориентироваться на качество из контеста.


# Задание (Coursera) - Сентимент-анализ отзывов на товары (простая версия)

Итак, вы уже немного познакомились с задачей. Теперь попробуйте поучаствовать в соревновании по анализу тональности текстов и побить бейзлайн (или вовсе выйти в топ участников).

В этом задании вам нужно воспользоваться опытом предыдущих недель, чтобы побить бейзлайн в соревновании по сентимент-анализу отзывов на товары на Kaggle Inclass:

https://inclass.kaggle.com/c/product-reviews-sentiment-analysis-light

В качестве ответа в этом задании вам нужно загрузить ноутбук с решением и скриншот вашего результата на leaderboard.

Убедитесь, что:

1) ход вашего решения задокументирован достаточно подробно для того, чтобы ваши сокурсники поняли, что вы делали и почему,

2) ваша команда в соревновании состоит только из вас и названа вашим логином на Сoursera, чтобы ваши сокурсники могли понять, что на скриншоте именно ваш результат

# Задание (kaggle) - Сентимент-анализ отзывов на товары (простая версия)

В этом соревновании вам предстоит прогнозировать по тексту отзыва его тональность: 1 - позитивная, 0 - негативная. В отличие от усложненной версии задачи, здесь вам не требуется самостоятельно собирать обучающую выборку - она есть в предоставляемых вам данных.

В качестве метрики качества используется accuracy. При валидации качества не забывайте, что ваш результат точно должен быть лучше, чем тривиальные ответы (всегда 0, всегда 1, случайный выбор класса).

Датасеты для загрузки:

products_sentiment_sample_submission.csv

products_sentiment_test.tsv

products_sentiment_train.tsv

В обучающей выборке на каждой строчке дано по одному тексту с классом (0 или 1), записанным через символ табуляции после текста.

In [102]:
import numpy as np
import pandas as pd

from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV
from sklearn.linear_model import LogisticRegression, SGDClassifier, RidgeClassifier
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

import re
import string

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk import pos_tag, word_tokenize  

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Python\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Python\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Python\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Python\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


#### Загрузка данных

In [103]:
#Загрузка данных
train = pd.read_table('products_sentiment_train.tsv', names=['text', 'target'])
test = pd.read_table('products_sentiment_test.tsv', index_col='Id')

In [104]:
#Взглянем на данные
print(train.head(10)) 
print(test.head(10))

print(train.shape)
print(test.shape)

                                                text  target
0          2 . take around 10,000 640x480 pictures .       1
1  i downloaded a trial version of computer assoc...       1
2  the wrt54g plus the hga7t is a perfect solutio...       1
3  i dont especially like how music files are uns...       0
4  i was using the cheapie pail ... and it worked...       1
5  you can manage your profile , change the contr...       1
6  11 ) minimal pause between songs - less than t...       1
7  from the internet , woodworking books , local ...       1
8  i loved this product when i first got it - gre...       1
9  whether you are using disposable or cloth diap...       1
                                                 text
Id                                                   
0   so , why the small digital elph , rather than ...
1   3/4 way through the first disk we played on it...
2   better for the zen micro is outlook compatibil...
3     6 . play gameboy color games on it with goboy .
4   l

In [105]:
#Загрузка примера файла ответов
sample_submission = pd.read_csv('products_sentiment_sample_submission.csv')
print(sample_submission.head(10))

   Id  y
0   0  0
1   1  1
2   2  0
3   3  1
4   4  0
5   5  1
6   6  0
7   7  1
8   8  0
9   9  1


In [106]:
#Загрузка списка стоп-слов
stop_words = nltk.corpus.stopwords.words('english')

#### Выполняем предобработку данных

In [107]:
# загружаем необходимый модуль для лемматизации
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

In [108]:
# функция для удаления цифр из текста
def hasNumb(inputString):
    return any(char.isdigit() for char in inputString)

In [109]:
# функции для токенизации и тегирования слов в текстах, удаления всех символов пунктуации и проведения лемматизации

def lemmatize_wtag(notlemmaword):
    for word, tag in pos_tag(word_tokenize(notlemmaword)):
        wntag = tag[0].lower()
        wntag = wntag if wntag in ['a', 'r', 'n', 'v'] else None
        if not wntag:
            lemma = word
        else:
            lemma = wordnet_lemmatizer.lemmatize(word, wntag)
        return lemma

pattern = re.compile('[%s]' % re.escape(string.punctuation))

def text_modified(to):
    to = pattern.sub('', to)
    words = [x.strip() for x in to.split() if not hasNumb(x)]
    result = []
    for word in words:
        word = lemmatize_wtag(word)
        result.append(word)      
    return ' '.join(result)

In [110]:
train['text_modified'] = train.text.apply(text_modified)
test['text_modified'] = test.text.apply(text_modified)

In [111]:
#Взглянем на подготовленные тексты обучающей выборки
train.head(10)

text  target  \
0          2 . take around 10,000 640x480 pictures .       1   
1  i downloaded a trial version of computer assoc...       1   
2  the wrt54g plus the hga7t is a perfect solutio...       1   
3  i dont especially like how music files are uns...       0   
4  i was using the cheapie pail ... and it worked...       1   
5  you can manage your profile , change the contr...       1   
6  11 ) minimal pause between songs - less than t...       1   
7  from the internet , woodworking books , local ...       1   
8  i loved this product when i first got it - gre...       1   
9  whether you are using disposable or cloth diap...       1   

                                       text_modified  
0                                take around picture  
1  i download a trial version of computer associa...  
2  the plus the be a perfect solution if you need...  
3  i dont especially like how music file be unstr...  
4  i be use the cheapie pail and it work ok until...  
5  you can manage your profile change the contras...  
6  minimal pause between song less than that on o...  
7  from the internet woodworking book local store...  
8  i love this product when i first get it great ...  
9  whether you be use disposable or cloth diaper ...

In [112]:
#Взглянем на подготовленные тексты тестовой выборки
test.head(10)

text  \
Id                                                      
0   so , why the small digital elph , rather than ...   
1   3/4 way through the first disk we played on it...   
2   better for the zen micro is outlook compatibil...   
3     6 . play gameboy color games on it with goboy .   
4   likewise , i 've heard norton 2004 professiona...   
5   mine was 2 weeks old and i chucked it in the t...   
6   i find it very stable and comfortable to use ,...   
7   styling / ergonomics : the keys are small , wh...   
8   at first i thought it is only a isolated incid...   
9                           - light , compact design    

                                        text_modified  
Id                                                     
0   so why the small digital elph rather than one ...  
1   way through the first disk we played on it nat...  
2     well for the zen micro be outlook compatibility  
3            play gameboy color game on it with goboy  
4   likewise i ve heard norton professional versio...  
5   mine be week old and i chucked it in the trash...  
6   i find it very stable and comfortable to use i...  
7   style ergonomics the key be small which genera...  
8   at first i thought it be only a isolated incid...  
9                                light compact design

#### Выбор классификатора
#### В качестве векторизатора - CountVectorizer

In [113]:
# Функция для выполнения кросс-валидации
def score(clf):
    scores = cross_val_score(clf, train.text_modified, train.target, cv=5)
    print("CLF score mean = {:.5f}, std = {:.5f}".format(scores.mean(), scores.std()))

In [114]:
# Сравниваем различные классификаторы
for classf in [RidgeClassifier, LogisticRegression, SGDClassifier, LinearSVC, MultinomialNB]:
    clf = make_pipeline(CountVectorizer(ngram_range=(1,2)), classf())
    score(clf)

CLF score mean = 0.77150, std = 0.00794
CLF score mean = 0.78300, std = 0.01114
CLF score mean = 0.76200, std = 0.00840
CLF score mean = 0.77749, std = 0.01432
CLF score mean = 0.78300, std = 0.01150


In [115]:
# Лучшими оказываются классификаторы LogisticRegression и MultinomialNB

#### Пробуем в качестве классификатора MultinomialNB (с подбором гиперпараметра alpha)
#### В качестве векторизатора - CountVectorizer

In [116]:
#Проведем подбор гиперпараметра alpha для классификатора MultinomialNB
for i in list(range(1, 201)):
    cv_score_nb = cross_val_score(make_pipeline(CountVectorizer(), 
                                               MultinomialNB(alpha=1+i*0.005)), train.text_modified, train.target, cv=5)
    print(0+i*0.01, cv_score_nb.mean())   

(0.01, 0.78450820942630894)
(0.02, 0.78450820942630894)
(0.03, 0.78450820942630894)
(0.04, 0.78350695629347678)
(0.05, 0.78350695629347678)
(0.06, 0.78400695629347683)
(0.07, 0.78400695629347683)
(0.08, 0.78300820317626985)
(0.09, 0.7825082031762699)
(0.1, 0.78200945005906297)
(0.11, 0.7820081969262308)
(0.12, 0.7820081969262308)
(0.13, 0.7820081969262308)
(0.14, 0.7820081969262308)
(0.15, 0.78250695004343773)
(0.16, 0.78250695004343773)
(0.17, 0.78300695004343779)
(0.18, 0.78300695004343779)
(0.19, 0.78350695004343773)
(0.2, 0.78350695004343773)
(0.21, 0.78400695004343768)
(0.22, 0.78400695004343768)
(0.23, 0.7845082031762699)
(0.24, 0.7845082031762699)
(0.25, 0.7845082031762699)
(0.26, 0.7845082031762699)
(0.27, 0.7845082031762699)
(0.28, 0.78500820317626974)
(0.29, 0.7845082031762699)
(0.3, 0.78450945005906281)
(0.31, 0.78450945005906281)
(0.32, 0.78500945005906286)
(0.33, 0.78500945005906286)
(0.34, 0.78500945005906286)
(0.35000000000000003, 0.78500945005906286)
(0.36, 0.7845094500

In [117]:
# Выбор лучшего классификатора и обучение: MultinomialNB с параметром alpha = 0.53
clf = make_pipeline(CountVectorizer(), MultinomialNB(alpha=0.53))
clf.fit(train.text_modified, train.target)

Pipeline(steps=[('countvectorizer', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)), ('multinomialnb', MultinomialNB(alpha=0.53, class_prior=None, fit_prior=True))])

In [118]:
# Подготовка файла с предсказаниями для загрузки на kaggle

In [119]:
predictions_NB = clf.predict(test.text_modified)

In [120]:
out_NB = pd.DataFrame(predictions_NB, index=test.index, columns=['y'])

In [121]:
out_NB.to_csv('submission_NB.csv') #результат на kaggle 0.79749 - пробуем дальше

#### Пробуем в качестве классификатора LogisticRegression
#### В качестве векторизатора - CountVectorizer

In [122]:
# Выбор классификатора и обучение: 
clf_LR = make_pipeline(CountVectorizer(ngram_range=(1,2)), LogisticRegression())
clf_LR.fit(train.text_modified, train.target)

Pipeline(steps=[('countvectorizer', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [123]:
predictions_LR = clf_LR.predict(test.text_modified)

In [124]:
out_LR = pd.DataFrame(predictions_LR, index=test.index, columns=['y'])

In [125]:
out_LR.to_csv('submission_LR.csv') #результат на kaggle 0.8000

#### Попробуем в качестве классификатора LinearSVC
#### В качестве векторизатора - TfidfVectorizer

In [126]:
# настраиваем список параметров, которые можно изменять в pipeline
param_grid = {'tfidfvectorizer__stop_words': ['english', stop_words], 
               'tfidfvectorizer__ngram_range': [(1, 1), (1, 2), (1, 3), (3, 5)],
              'tfidfvectorizer__analyzer': ['word', 'char'],
              'tfidfvectorizer__min_df': [0, 1, 2]}

# валидация по трём фолдам
cv = KFold(n_splits=3, shuffle=True, random_state=42)

#пайплайн
model = make_pipeline(TfidfVectorizer(), LinearSVC())

#перебор параметров для поиска наилучшего результата
rsearch = GridSearchCV(model, param_grid, scoring="accuracy", n_jobs=1, cv=cv, verbose=1)

#обучение модели на обучающей выборке
rsearch.fit(train.text, train.target)
print (rsearch.best_score_)
print (rsearch.best_params_)

Fitting 3 folds for each of 48 candidates, totalling 144 fits


[Parallel(n_jobs=1)]: Done 144 out of 144 | elapsed:  1.0min finished


0.7785
{'tfidfvectorizer__analyzer': 'char', 'tfidfvectorizer__ngram_range': (3, 5), 'tfidfvectorizer__min_df': 0, 'tfidfvectorizer__stop_words': 'english'}


In [127]:
predictions2 = rsearch.predict(test.text)

In [128]:
out2 = pd.DataFrame(predictions2, index=test.index, columns=['y'])

In [129]:
out2.to_csv('submission2.csv') #результат на kaggle 0.80749 - бейзлайн побит!!!